# Εισάγουμε τις απαραίτητες βιβλιοθήκες

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import ast
import time
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
import os

## Εισάγουμε τα δεδομένα και κρατάμε τις στήλες που χρειαζόμαστε

In [ ]:
skai_df = pd.read_csv("https://raw.githubusercontent.com/stavrouk/Stavrou_Teliki_Ergasia/main/Scraping/skai_emvolio.csv")

In [ ]:
skai_df= skai_df[['title' , 'date' , 'summary' , 'body']]

In [58]:
skai_df.head()

In [ ]:
len(skai_df)

1795

In [ ]:
star_df = pd.read_csv("https://raw.githubusercontent.com/stavrouk/Stavrou_Teliki_Ergasia/main/Scraping/star_emvolio.csv")

In [ ]:
star_df= star_df[['title' , 'date' , 'summary' , 'body']]

In [59]:
star_df.head()

In [ ]:
len(star_df)

509

## Αλλάζουμε το date σε datetime και δημιουργούμε 3 νέες στήλες (ημέρα, μήνας, έτος)

In [ ]:
skai_df['date'] = pd.to_datetime(skai_df['date'], format='%d/%m/%Y - %H:%M')
skai_df['date']

0      2021-04-14 01:09:00
1      2021-05-19 16:04:00
2      2021-03-05 20:10:00
3      2021-02-01 09:34:00
4      2020-12-31 09:46:00
               ...        
1790   2021-01-07 18:38:00
1791   2021-02-18 13:14:00
1792   2021-03-16 10:03:00
1793   2021-03-24 17:53:00
1794   2021-03-12 17:42:00
Name: date, Length: 1795, dtype: datetime64[ns]

In [ ]:
#Αλλάζουμε το format του έτους στο date για να μπορεί να αναγνωστεί ως %Y και να μετατραπεί σε datetime
star_df['date'] = star_df['date'].str.replace("20,", "2020,")
star_df['date'] = star_df['date'].str.replace("21,", "2021,")

In [ ]:
star_df['date'] = pd.to_datetime(star_df['date'], format='%d.%m.%Y, %H:%M')
star_df['date']

0     2021-03-01 19:02:00
1     2021-02-19 10:53:00
2     2021-01-24 15:00:00
3     2021-01-04 16:09:00
4     2021-03-19 19:38:00
              ...        
504   2021-01-04 12:00:00
505   2021-06-16 10:13:00
506   2021-06-04 11:32:00
507   2021-05-31 19:51:00
508   2021-03-04 13:37:00
Name: date, Length: 509, dtype: datetime64[ns]

In [ ]:
skai_df['year'] = pd.DatetimeIndex(skai_df['date']).year
skai_df['month'] = pd.DatetimeIndex(skai_df['date']).month
skai_df['day'] = pd.DatetimeIndex(skai_df['date']).day

In [ ]:
star_df['year'] = pd.DatetimeIndex(star_df['date']).year
star_df['month'] = pd.DatetimeIndex(star_df['date']).month
star_df['day'] = pd.DatetimeIndex(star_df['date']).day

## Καθαρίζουμε τα άρθρα από περιττά στοιχεία και διαλέγουμε τα άρθρα που δημοσιεύτηκαν το 2021

In [ ]:
skai_df['body'] = skai_df['body'].str.replace(r'https?:\/\/.*[\r\n]*'," ")

In [ ]:
star_df['body'] = star_df['body'].str.replace(r'https?:\/\/.*[\r\n]*'," ")

In [ ]:
star_df['body'] = star_df['body'].str.replace('googletag',' ').str.replace('ad',' ').str.replace('gpt',' ').str.replace('function',' ').str.replace('div',' ').str.replace('display',' ').str.replace('cmd',' ').str.replace('push',' ').str.replace('1618837763216',' ')

In [ ]:
star_df['body'] = star_df['body'].str.replace('\n'," ")
star_df['body'] = star_df['body'].str.replace('\xa0'," ")
skai_df['body'] = skai_df['body'].str.replace('\n'," ")

In [ ]:
star_df = star_df[star_df['year'] == 2021]

In [ ]:
skai_df = skai_df[skai_df['year'] == 2021]

In [60]:
skai_df.head()

In [ ]:
len(skai_df)

1659

In [61]:
star_df.head()

In [ ]:
len(star_df)

469

# Ορίζουμε την ημερομηνία ως index και δημιουργούμε τα πρώτα γραφήματα

In [ ]:
skai_df.set_index('date' , inplace=True)
star_df.set_index('date' , inplace=True)

In [ ]:
#Κάνουμε sort για να φαίνονται τα άρθρα ανάλογα με την ημερομηνία δημοσίευσης 
skai_df = skai_df.sort_index()
star_df = star_df.sort_index()

In [ ]:
#Ο αριθμός των δημοσιευμένων άρθρων για τα εμβόλια, ανά μήνα στο Skai
skai_df['2021-01': ].value_counts('month')

month
1    383
2    312
3    306
5    250
4    240
6    168
dtype: int64

In [ ]:
#Ο αριθμός των δημοσιευμένων άρθρων για τα εμβόλια, ανά μήνα στο Star
star_df['2021-01': ].value_counts('month')

month
5    109
4     97
1     82
3     66
6     64
2     51
dtype: int64

In [50]:
#Γράφημα με τον αριθμό των δημοσιευμένων άρθρων για τα εμβόλια, ανά μήνα στο Skai
skai_df['2021-01': ].value_counts('month').plot(kind= 'bar', figsize= (10,6))

In [51]:
#Γράφημα με τον αριθμό των δημοσιευμένων άρθρων για τα εμβόλια, ανά μήνα στο Star
star_df['2021-01': ].value_counts('month').plot(kind= 'bar', figsize= (10,6), color= 'brown')

In [ ]:
#Ο μέσος όρος των δημοσιευμένων άρθρων για τα εμβόλια, ανά μήνα στο Skai
skai_df['month'].value_counts().sort_values(ascending= False).mean()

276.5

In [ ]:
#Ο μέσος όρος των δημοσιευμένων άρθρων για τα εμβόλια, ανά μήνα στο Star
star_df['month'].value_counts().sort_values(ascending= False).mean()

78.16666666666667

## Φτιάχνουμε το WordCloud από τα κείμενα των τίτλων και των περιγραφών

In [ ]:
#Δημιουργούμε νέα στήλη με το κείμενο του τίτλου και της περιγραφής, των άρθρων
skai_df["text"] = skai_df["title"] + skai_df["summary"]

In [ ]:
#Δημιουργούμε νέα στήλη με το κείμενο του τίτλου και της περιγραφής, των άρθρων
star_df["text"] = star_df["title"] + star_df["summary"]

In [ ]:
from wordcloud import WordCloud, STOPWORDS

In [ ]:
#Καθαρίζουμε το κείμενο από περιττούς χαρακτήρες
skaitext = skai_df['text'].replace("amp", ' ').str.replace('AMP',' ').str.replace('Amp',' ').str.replace('links',' ').str.replace("RT", " ").str.replace('@',' ').str.replace('http',' ').str.replace('https',' ').str.replace('rt',' ').str.cat(sep = ' ')

In [ ]:
#Καθαρίζουμε το κείμενο από περιττούς χαρακτήρες
startext = star_df['text'].replace("amp", ' ').str.replace('AMP',' ').str.replace('Amp',' ').str.replace('links',' ').str.replace("RT", " ").str.replace('@',' ').str.replace('http',' ').str.replace('https',' ').str.replace('rt',' ').str.cat(sep = ' ')

In [62]:
skaitext

In [63]:
startext

In [ ]:
#Εισάγουμε την απαραίτητη βιβλιοθήκη και το ελληνικό λεξικό
import spacy

In [ ]:
#!python -m spacy download el_core_news_sm

In [ ]:
nlp = spacy.load('el_core_news_sm')

In [64]:
list(nlp.Defaults.stop_words)[0:20]

In [ ]:
full_doc1 = nlp(skaitext)

In [ ]:
full_doc2 = nlp(startext)

In [ ]:
clear_text1 = ' '.join(token.lemma_ for token in full_doc1)

In [ ]:
clear_text2 = ' '.join(token.lemma_ for token in full_doc2)

In [ ]:
#Φτιάχνουμε το Wordcloud του Skai
wordcloud = WordCloud(
    stopwords = nlp.Defaults.stop_words,
    width = 2000,
    height = 1000,
    background_color = 'black'
 ).generate(clear_text1)
fig = plt.figure(
    figsize = (40, 30),
    facecolor = 'k',
    edgecolor = 'k')
plt.imshow(wordcloud, interpolation = 'bilinear')
plt.axis('off')
plt.tight_layout(pad=0)
plt.show()

In [ ]:
#Φτιάχνουμε το Wordcloud του Star
wordcloud = WordCloud(
    stopwords = nlp.Defaults.stop_words,
    width = 2000,
    height = 1000,
    background_color = 'black'
 ).generate(clear_text2)
fig = plt.figure(
    figsize = (40, 30),
    facecolor = 'k',
    edgecolor = 'k')
plt.imshow(wordcloud, interpolation = 'bilinear')
plt.axis('off')
plt.tight_layout(pad=0)
plt.show()

## Δημιουργούμε διαγράμματα με τις πιο συχνές λέξεις στο βασικό κείμενο των άρθρων

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()

In [ ]:
cv = CountVectorizer( stop_words= nlp.Defaults.stop_words, max_features=20)
count_vector1 = cv.fit_transform(skai_df['body'])
results1 = pd.DataFrame(count_vector1.toarray(), columns=cv.get_feature_names()) 
results1.head()


,000,19,astrazeneca,covid,pfizer,δήλωσε,δόσεις,δόση,δύο,εμβολίου,εμβολίων,εμβολιασμού,εμβόλια,εμβόλιο,ετών,πηγή,σήμερα,σύμφωνα,υγείας,χώρες
0,1,1,0,1,0,0,0,0,1,0,0,0,0,1,0,1,0,0,1,0
1,2,0,1,0,0,0,2,0,0,2,0,1,0,1,0,1,0,0,0,0
2,0,2,1,2,1,0,0,0,1,0,0,0,2,1,0,1,1,1,0,0
3,5,0,0,1,0,0,0,0,1,1,0,0,1,9,1,1,2,0,3,2
4,0,2,0,2,2,1,1,0,0,1,1,0,1,5,0,1,0,0,2,3


In [ ]:
cv = CountVectorizer( stop_words= nlp.Defaults.stop_words, max_features=20)
count_vector2 = cv.fit_transform(star_df['body'])
results2 = pd.DataFrame(count_vector2.toarray(), columns=cv.get_feature_names()) 
results2.head()


,19,astrazeneca,covid,johnson,pfizer,διαβάστε,δόσεις,δόση,ειδήσεις,ελλάδα,εμβολίου,εμβολίων,εμβολιασμό,εμβόλια,εμβόλιο,ετών,κορωνοϊού,κόσμο,ραντεβού,υγείας
0,0,2,0,0,4,1,3,0,1,1,6,3,0,4,8,0,3,1,0,1
1,0,0,0,0,3,2,0,0,2,1,1,1,0,0,2,0,2,1,0,1
2,0,0,0,0,0,1,0,1,1,2,1,0,1,0,0,0,2,1,0,2
3,0,0,1,0,1,1,0,0,1,1,0,1,1,0,4,0,1,2,0,1
4,1,2,1,0,0,1,0,0,1,1,1,0,0,0,8,0,1,1,0,1


In [ ]:
results1.sum(axis =0).sort_values(ascending = False)

εμβόλιο        3458
εμβολίου       2345
covid          1996
19             1871
εμβόλια        1820
πηγή           1627
υγείας         1561
astrazeneca    1493
εμβολίων       1469
δόσεις         1291
σήμερα         1189
δόση           1149
ετών           1091
pfizer         1083
δύο            1081
000             977
χώρες           972
δήλωσε          966
σύμφωνα         961
εμβολιασμού     959
dtype: int64

In [52]:
#Φτιάχνουμε γράφημα με τις 15 συχνότερες λέξεις στο κείμενο του Skai
results1.sum(axis =0).sort_values(ascending = False).head(15).plot(kind= 'bar', figsize=(10,5))

In [ ]:
results2.sum(axis =0).sort_values(ascending = False)

εμβόλιο        1764
astrazeneca     974
εμβολίου        796
ελλάδα          622
ετών            600
διαβάστε        553
κόσμο           539
δόση            520
εμβόλια         493
ειδήσεις        493
pfizer          471
johnson         463
εμβολιασμό      418
ραντεβού        402
covid           372
19              368
κορωνοϊού       368
δόσεις          341
υγείας          339
εμβολίων        329
dtype: int64

In [53]:
#Φτιάχνουμε γράφημα με τις 15 συχνότερες λέξεις στο κείμενο του Star
results2.sum(axis =0).sort_values(ascending = False).head(15).plot(kind= 'bar', figsize=(10,5), color= 'brown')

## Δημιουργούμε διαγράμματα με τα πιο συχνά διγράμματα στο βασικό κείμενο των άρθρων

In [ ]:
cv = CountVectorizer( stop_words= nlp.Defaults.stop_words, max_features=20, ngram_range=(2, 2) )
count_vector1 = cv.fit_transform(skai_df['body'])
results1 = pd.DataFrame(count_vector1.toarray(), columns=cv.get_feature_names()) 
results1.head()

,covid 19,html5 video,johnson johnson,pfizer biontech,sars cov,skai gr,view this,απε μπε,δεύτερη δόση,δόσεις εμβολίου,δόση εμβολίου,εκατομμύρια δόσεις,εμβολίου astrazeneca,εμβόλιο astrazeneca,εμβόλιο pfizer,ηνωμένο βασίλειο,κυριάκος μητσοτάκης,πηγή skai,πηγή απε,υπουργός υγείας
0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0
1,0,0,0,0,0,0,0,0,0,2,0,0,0,1,0,0,0,0,0,0
2,2,0,0,1,0,0,0,1,0,0,0,0,0,0,0,2,0,0,1,0
3,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0
4,2,0,0,2,0,0,0,1,0,1,0,0,0,0,1,1,0,0,1,0


In [ ]:
cv = CountVectorizer( stop_words= nlp.Defaults.stop_words, max_features=20, ngram_range=(2, 2) ) 
count_vector2 = cv.fit_transform(star_df['body'])
results2 = pd.DataFrame(count_vector2.toarray(), columns=cv.get_feature_names()) 
results2.head()

,60 ετών,covid 19,gov gr,johnson johnson,pfizer biontech,δελτίο ειδήσεων,δεύτερη δόση,διαβάστε lifestyle,διαβάστε ειδήσεις,δόσεις εμβολίου,δόση εμβολίου,ειδήσεις ελλάδα,ειδήσεων star,ελλάδα κόσμο,εμβολίου astrazeneca,εμβόλιο astrazeneca,εμβόλιο johnson,εμβόλιο pfizer,εμβόλιο κορωνοϊού,ηλικιακή ομάδα
0,0,0,0,0,2,0,0,0,1,2,0,1,0,1,0,1,0,1,0,0
1,0,0,0,0,2,0,0,0,2,0,0,1,0,1,0,0,0,2,0,0
2,0,0,0,0,0,0,0,0,1,0,1,1,0,1,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,1,0,0
4,0,1,0,0,0,0,0,0,1,0,0,1,0,1,0,1,0,0,1,0


In [ ]:
results1.sum(axis =0).sort_values(ascending = False)

covid 19                1753
απε μπε                  881
πηγή απε                 850
skai gr                  638
πηγή skai                620
εμβόλιο astrazeneca      491
pfizer biontech          419
sars cov                 378
δόση εμβολίου            370
johnson johnson          369
δεύτερη δόση             353
δόσεις εμβολίου          316
εκατομμύρια δόσεις       276
υπουργός υγείας          267
κυριάκος μητσοτάκης      264
εμβολίου astrazeneca     250
εμβόλιο pfizer           208
view this                206
ηνωμένο βασίλειο         201
html5 video              200
dtype: int64

In [54]:
#Φτιάχνουμε γράφημα με τα 15 συχνότερα διγράμματα στο κείμενο του Skai
results1.sum(axis =0).sort_values(ascending = False).head(15).plot(kind= 'bar', figsize= (10,6))

In [ ]:
results2.sum(axis =0).sort_values(ascending = False)

διαβάστε ειδήσεις       473
ελλάδα κόσμο            469
ειδήσεις ελλάδα         425
εμβόλιο astrazeneca     411
covid 19                332
johnson johnson         230
δόση εμβολίου           168
δεύτερη δόση            165
ειδήσεων star           135
εμβόλιο pfizer          128
pfizer biontech         124
εμβόλιο johnson         110
δελτίο ειδήσεων         109
εμβολίου astrazeneca    101
ηλικιακή ομάδα           85
60 ετών                  85
διαβάστε lifestyle       77
δόσεις εμβολίου          75
εμβόλιο κορωνοϊού        73
gov gr                   72
dtype: int64

In [55]:
#Φτιάχνουμε γράφημα με τα 15 συχνότερα διγράμματα στο κείμενο του Star
results2.sum(axis =0).sort_values(ascending = False).head(15).plot(kind= 'bar', figsize= (10,6), color= 'brown')

## Βρίσκουμε την ομοιότητα των κειμένων

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
tfidf_vectorizer1 = TfidfVectorizer(stop_words= nlp.Defaults.stop_words, max_features=500)
tfidf_matrix_train1 = tfidf_vectorizer1.fit_transform(skai_df['body']) 

In [ ]:
tfidf_vectorizer2 = TfidfVectorizer(stop_words= nlp.Defaults.stop_words, max_features=500)
tfidf_matrix_train2 = tfidf_vectorizer2.fit_transform(star_df['body'])  

In [69]:
results1 = pd.DataFrame(tfidf_matrix_train1.toarray(), columns=tfidf_vectorizer1.get_feature_names()) 
results1.head()

In [70]:
results2 = pd.DataFrame(tfidf_matrix_train2.toarray(), columns=tfidf_vectorizer2.get_feature_names()) 
results2.head()

In [67]:
from sklearn.metrics.pairwise import cosine_similarity

print("cosine scores ==> ",cosine_similarity(tfidf_matrix_train1, tfidf_matrix_train1))

In [68]:
print("cosine scores ==> ",cosine_similarity(tfidf_matrix_train2, tfidf_matrix_train2))

In [ ]:
arr1 = cosine_similarity(tfidf_matrix_train1, tfidf_matrix_train1)

In [ ]:
arr2 = cosine_similarity(tfidf_matrix_train2, tfidf_matrix_train2)

## Δημιουργούμε τα δύο Heatmap

In [ ]:
import seaborn as sb
from matplotlib.colors import ListedColormap

In [ ]:
#Κάνουμε dataframe τις δύο λίστες με τα cosine scores
plt_df1 = pd.DataFrame(arr1)
plt_df2 = pd.DataFrame(arr2)

In [65]:
plt_df1.head()

In [66]:
plt_df2.head()

In [56]:
# Φτιάχνουμε το Heatmap του Skai

# Δημιουργούμε έναν πίνακα, ορίζοντας το index κάθε γραμμής από το df του plt_df1 (άρθρα του Skai).
# Στη συνέχεια βρίσκουμε τα 20 άρθρα με την μεγαλύτερη ομοιότητα και τα αθροίζουμε.
scores1 = []
for index, row in plt_df1.iterrows():
  scores1.append({ 'i' : index, 'similarity_score': row.sort_values(ascending = False).head(20).sum()})

# Ορίζουμε ως dataframe την συνθήκη που κάναμε πριν
scores1 = pd.DataFrame(scores1)

# Παίρνουμε τα 20 άρθρα από το dataframe scores1
top20scores1 = scores1.sort_values('similarity_score', ascending=False).head(20)

# Στη συνέχεια παίρουμε τα άρθρα, το index και τις στήλες από τα 20 άρθρα με τη μεγαλύτερη ομοιότητα.
top20similar1 =  plt_df1.loc[list(top20scores1['i'])][list(top20scores1.head(20)['i'])]

# Δημιουργούμε το Heatmap
plt.figure(figsize=(15,12))
sb.heatmap(top20similar1, annot=True, cmap=plt.cm.Blues)
plt.title('Ομοιότητα άρθρων Skai', fontsize=13)
plt.show()

In [57]:
# Φτιάχνουμε το Heatmap του Star

# Κάνουμε την ίδια διαδικασία με πριν, αλλάζοντας το dataframe για να πάρουμε τα άρθρα του Star
scores2 = []
for index, row in plt_df2.iterrows():
  scores2.append({ 'i' : index, 'similarity_score': row.sort_values(ascending = False).head(20).sum()})

scores2 = pd.DataFrame(scores2)

top20scores2 = scores2.sort_values('similarity_score', ascending=False).head(20)

top20similar2 =  plt_df2.loc[list(top20scores2['i'])][list(top20scores2.head(20)['i'])]

plt.figure(figsize=(15,12))
sb.heatmap(top20similar2, annot=True, cmap=plt.cm.OrRd)
plt.title('Ομοιότητα άρθρων Star', fontsize=13)
plt.show()